# 검색증강생성(RAG)을 활용한 챗봇 구현

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다

In [ ]:
import boto3
from utils.ssm import parameter_store

region=boto3.Session().region_name
pm = parameter_store(region)

domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

region_name = 'us-west-2'
retry_config = Config(
        region_name=region_name,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
boto3_bedrock = boto3.client("bedrock-runtime", region_name=region_name, config=retry_config)

llmchat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=boto3_bedrock,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"]
    }
)

from langchain.embeddings import BedrockEmbeddings

llmemb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-g1-text-02"
)
dimension = 1536

In [ ]:
from utils.rag import qa_chain
from utils.rag import prompt_repo, show_context_used
from utils.rag import retriever_utils, OpenSearchHybridSearchRetriever
from opensearchpy import OpenSearch, RequestsHttpConnection

http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
        hosts=[
            {'host': opensearch_domain_endpoint.replace("https://", ""),
             'port': 443
            }
        ],
        http_auth=http_auth, 
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection
    )

In [ ]:
index_name = "sample_index"
opensearch_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llmchat, 
    llm_emb=llmemb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[1.0, 0.0], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=False, 
    parent_document = False, # enable parent document
    
    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [ ]:
system_prompt = prompt_repo.get_system_prompt()
# 기본 프롬프트 템플릿을 불러와서 활용
print(system_prompt)

In [ ]:
qa = qa_chain(
    llm_text=llmchat,
    retriever=opensearch_retriever,
    system_prompt=system_prompt,
    return_context=True,
    verbose=False
)

In [ ]:
query = "한약재 투약비용도 보험금을 지급받을 수 있을까요?"

response, contexts = qa.invoke(
    query = query
)

print("\n\n\n==============아래는 위 답변에 사용된 컨텍스트입니다==============\n")
show_context_used(contexts)

### RAG 기능을 Streamlit 애플리케이션에 구현

앞서 저장했던 OpenSearch의 벡터 인덱스(`sample_pdf`)에서 질문에 맞는 컨텍스트를 찾아옵니다.

OpenSearch에서 컨텍스트를 찾기 위한 방법으로 **1) 벡터(Semantic) 검색**, **2) 텍스트(Lexical) 검색**을 사용할 수 있습니다.

RAG에서 자연어 검색을 위해 기본적으로 벡터 검색을 사용하지만, 특정 검색어에서는 텍스트 검색에서 더 적합한 컨텍스트를 찾아냅니다.

이를 위해, **벡터 검색과 텍스트 검색을 앙상블**해서 사용하는 **하이브리드 검색**을 최적화 기법으로 사용하기도 합니다. 

아래는 `Hybrid-RAG`를 검색 옵션으로 선택했을 때 **벡터 검색 - 0.51, 텍스트 검색 - 0.49**의 가중치로 컨텍스트를 얻는 기능을 포함합니다.

In [ ]:
!cp ./chat-rag-opensearch-hybrid.py ../demo-app.py

# 예상 질문 목록

## sample1_ko.pdf : 해외여행보험 보통약관
#### 특별비용담보 특별약관에서 회사가 보상하는 비용의 범위는?
#### 한약재 투약비용도 보험금을 지급받을 수 있어?
#### 외교관의 보험가입금액에 대해 알고싶은데, 피보험자가 자녀인 경우, 사망.후유장해에 대한 자녀 1인당 보험가입금액이 얼마야?
#### 계약자가 사망보험을 가입한 후에, 보험금수익자를 변경하려면 어떤 절차를 따라야해?
#### 인질구조비용 특별약관에 가입하면, 인질상태에 놓였을 때 어떤 비용을 보상받을 수 있어?

## sample2_ko.pdf : 생성AI 산업 생태계 현황과 과제
#### Meta의 생성형 AI 전략에 대해 알려주세요
#### LG 전자는 어떤 생성형 AI 모델을 사용 중인지 알려주세요
#### 10년 후에 생성 AI 시장은 지금보다 얼마나 커질까요?
#### 클라우드에서 엔비디아 GPU의 점유율은 얼마나 될까?
#### 아마존의 생성형 AI 전략이 뭐야?
#### 삼성전자의 생성형 AI 전략이 뭐야?

## Model_Card_Llama_3.pdf : Meta LLAMA 3 모델 카드
#### Llama3는 언제, 어떤 라이센스로 배포가 되었나요?
#### Llama3는 언제까지의 최신정보를 갖고있나요? 그리고 어떤 종류의 데이터를 얼마나 많이 학습했나요?

## Model_Card_EEVE_LLM.pdf : EEVE LLM 모델 카드
#### EEVE 모델은 어떤 모델을 기반으로 파인튜닝 됐어?
#### EEVE 모델의 가장 특징적인 장점이 뭐야?